# S2W6D6: 深入工程内核 (Hand-written Dynamic Padding)

Day 6 的核心是 **“手写 `collate_fn`”**。这和 Day 4 的“调用现成类”是完全不同的层级：

  * **Day 4 (调用)**: 是你会**用**工具。
  * **Day 6 (手写)**: 是你能**造**工具。这是面试官考察你 pytorch 工程底功的**超高频考题**。

既然我们要在 Week 7 能够随心所欲地控制模型，今天我们就必须把这个“黑盒”拆开，自己手写一个动态 Padding 函数。

**🎯 核心目标**:

1.  不依赖 `transformers` 库的 Collator，仅用 `torch` 实现 **动态 Padding**。
2.  理解 `DataLoader` 是如何把一堆散乱的样本（List of Dicts）打包成一个规整的 Batch（Dict of Tensors）的。

## 1 📚 理论：DataLoader 的幕后黑手 `collate_fn`

当你写 `for batch in dataloader:` 时，PyTorch 内部发生了什么？

1.  **Sampler**: 决定取哪些索引（比如 `[0, 5, 2]`）。
2.  **Dataset**: 根据索引把数据拿出来，得到一个**列表**：

In [1]:
raw_batch = [
    {"input_ids": [101, 1, 2, 102], "label": 1},  # 长度 4
    {"input_ids": [101, 3, 4, 5, 6, 102], "label": 0}, # 长度 6
    {"input_ids": [101, 7, 102], "label": 1}      # 长度 3
]

3.  **`collate_fn` (核心)**: 它的任务是把上面这个**参差不齐的列表**，变成一个**整齐的 Tensor**。
      * **挑战**: 必须找出最大长度 (6)，把其他的补齐。

## 2 💻 代码实战：手撕 `collate_fn`

我们要实现一个函数，它接收 `raw_batch`，返回 `processed_batch`。

### 2.1 准备模拟数据

为了专注于逻辑，我们先造一点数据，不依赖外部文件。

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

# 1. 定义一个简单的 Dataset
class ToyDataset(Dataset):
    def __init__(self):
        # 模拟 3 条长短不一的数据
        self.data = [
            {"input_ids": [101, 2769, 102], "label": 1},           # Len: 3
            {"input_ids": [101, 2769, 3221, 511, 102], "label": 0}, # Len: 5
            {"input_ids": [101, 102], "label": 1}                  # Len: 2
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 注意：这里返回的是普通的 List，不是 Tensor
        return self.data[idx]

# 实例化
dataset = ToyDataset()
print("单条样本:", dataset[0])

单条样本: {'input_ids': [101, 2769, 102], 'label': 1}


### 2.2 核心任务：手写整理函数 (Interview Core)

请仔细阅读这段代码，这是面试时的**白板题标准答案**。

In [3]:
def my_collate_fn(batch):
    """
    输入 batch: 一个包含 N 个字典的 List (来自 dataset.__getitem__)
    [
      {'input_ids': [101, ...], 'label': 1},
      {'input_ids': [101, ...], 'label': 0}
    ]
    """

    # --- Step 1: 拆解数据 ---
    # 把 input_ids 和 labels 分别拿出来放到两个列表里
    # item['input_ids'] 此时是 list，我们需要转成 tensor 才能用 pad_sequence
    input_ids_list = [torch.tensor(item['input_ids']) for item in batch]
    labels_list = [item['label'] for item in batch]

    # --- Step 2: 动态 Padding (核心) ---
    # torch.nn.utils.rnn.pad_sequence 是神器
    # batch_first=True: 返回 [Batch, Max_Len] (否则是 [Max_Len, Batch])
    # padding_value=0: 用 0 填充
    input_ids_padded = pad_sequence(input_ids_list, batch_first=True, padding_value=0)

    # --- Step 3: 构造 Attention Mask (手动) ---
    # 逻辑：input_ids > 0 的地方是 1，等于 0 的地方是 0
    # 这一步非常巧妙，不用写循环
    attention_mask = (input_ids_padded > 0).long()

    # --- Step 4: 处理 Labels ---
    # 简单的堆叠即可
    labels = torch.tensor(labels_list)

    # --- Step 5: 打包返回 ---
    return {
        "input_ids": input_ids_padded,
        "attention_mask": attention_mask,
        "labels": labels
    }

# --- 验证一下 ---
# 模拟 DataLoader 取出的 raw_batch
raw_batch = [dataset[0], dataset[1], dataset[2]]
processed = my_collate_fn(raw_batch)

print("--- 手写 Collate 结果 ---")
print("Input IDs Shape:", processed['input_ids'].shape) # 应该是 [3, 5]
print("Input IDs:\n", processed['input_ids'])
print("Attention Mask:\n", processed['attention_mask'])
# 观察：短的句子后面是不是补了 0？Mask 对应的位置是不是 0？

--- 手写 Collate 结果 ---
Input IDs Shape: torch.Size([3, 5])
Input IDs:
 tensor([[ 101, 2769,  102,    0,    0],
        [ 101, 2769, 3221,  511,  102],
        [ 101,  102,    0,    0,    0]])
Attention Mask:
 tensor([[1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0]])


### 2.3 放入 DataLoader 验证

现在我们将它集成到真正的 DataLoader 中，模拟 Batch 训练。

In [ ]:
# 必须设置 shuffle=True 才能看到不同 batch 长度变化的效果 (虽然这里数据少可能不明显)
dataloader = DataLoader(
    dataset, 
    batch_size=2, 
    collate_fn=my_collate_fn, # 传入我们手写的函数
    shuffle=True
)

print("\n--- DataLoader 动态验证 ---")
for i, batch in enumerate(dataloader):
    print(f"Batch {i} Input Shape: {batch['input_ids'].shape}")
    # 重点观察：
    # 如果抽到了那条长度为 5 的数据，Shape 就是 [2, 5]
    # 如果只抽到了长度 3 和 2 的数据，Shape 就会变成 [2, 3] (这就是动态!)


--- DataLoader 动态验证 ---
Batch 0 Input Shape: torch.Size([2, 5])
Batch 1 Input Shape: torch.Size([1, 3])


## 3\. 💥 面试深挖 (Deep Dive)

**Q: `pad_sequence` 是 PyTorch 的函数，如果面试官不让你用这个工具，让你纯手写逻辑，怎么办？**

**A**: 其实也不难，逻辑如下（可以作为你的备选方案）：

In [5]:
def hard_core_collate(batch):
    # 1. 找出当前 batch 的最大长度
    max_len = max([len(item['input_ids']) for item in batch])
    
    batch_input_ids = []
    batch_masks = []
    batch_labels = []
    
    for item in batch:
        ids = item['input_ids']
        current_len = len(ids)
        
        # 2. 计算需要补多少个 0
        pad_len = max_len - current_len
        
        # 3. 拼接
        padded_ids = ids + [0] * pad_len
        mask = [1] * current_len + [0] * pad_len
        
        batch_input_ids.append(padded_ids)
        batch_masks.append(mask)
        batch_labels.append(item['label'])
        
    return {
        "input_ids": torch.tensor(batch_input_ids),
        "attention_mask": torch.tensor(batch_masks),
        "labels": torch.tensor(batch_labels)
    }

*Day 4 我们用的 `DataCollatorWithPadding` 实际上就是把上面这段逻辑封装好了，甚至做了更极致的优化（Rust 实现）。*

## 4\. ⚔️ 每日算法题 (LeetCode Daily)

既然我们今天延后了复习，那算法题依然要坚持。
配合今天的“动态容器”主题，我们做那道经典的 **双指针** 题目（之前提到过，今天正式做）。

### 🎯 题目: LeetCode 11. 盛最多水的容器 (Container With Most Water)

  * **难度**: Medium
  * **链接**: [LeetCode 11](https://leetcode.cn/problems/container-with-most-water/)
  * **描述**: 给你 `height` 数组，找出两条线，使得它们与 x 轴围成的容器能装最多的水。
  * **核心**: **贪心 + 双指针**。
      * 面积 = $(right - left) \times \min(height[left], height[right])$
      * **策略**: 总是移动高度较**小**的那根柱子。（因为宽度在减小，如果高度还不变大，面积肯定变小）。

## ✅ Day 6 任务清单

1.  **运行代码**: 必须亲手跑通 `my_collate_fn`。
2.  **观察动态性**: 确认不同的 batch 出来的 shape 是不一样的（比如一个是 `[2, 5]`，一个是 `[2, 3]`）。
3.  **LeetCode**: 完成 LeetCode 11。